En este notebook se carga el modelo desde Google Drive y se ejecuta la interfaz para realizar las inferencias

🔹 Listado de artistas obtenidos del dataset de entrenamiento

In [1]:
artist_names = [
    'Andrés Calamaro',
    'Chaqueño Palavecino',
    'Fito Páez',
    'Horacio Guarany',
    'Jairo',
    'Juan Gabriel',
    'La Barra',
    'Leo Dan',
    'Litto Nebbia',
    'Los Chalchaleros',
    'Los Palmeras',
    'Mercedes Sosa',
    'Pablo Milanés',
    'Palito Ortega',
    'Raphael',
    'Rocío Dúrcal',
    'Sabroso',
    'Sandro',
    'Ulises Bueno',
    'Víctor Heredia'
]


🔹 Carga del modelo desde Google Drive

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from google.colab import drive
import os
import torch

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
else:
    print("Drive ya está montado")

# Ruta base en Google Drive donde se guarda el modelo
BASE_PATH = "/content/drive/MyDrive/ProyectoFinal"
MODEL_PATH = f"{BASE_PATH}/FineTuning/DeepESP_gpt2-spanish/full_fine_tuning_v4"

print("Cargando modelo reentrenado desde Google Drive...")
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = "<pad>"
tokenizer.bos_token = "<bos>"
tokenizer.eos_token = "<eos>"
model = GPT2LMHeadModel.from_pretrained(MODEL_PATH)

# Mover el modelo al dispositivo
model = model.to(device)
print(f"Modelo en: {next(model.parameters()).device}")

# Verificar que los tokens especiales estén correctamente configurados:
print(f"Tokens especiales: {tokenizer.special_tokens_map}")
print(f"ID de <lyr>: {tokenizer.encode('<lyr>')}")
print(f"ID de <art>: {tokenizer.encode('<art>')}")
print(f"ID de <eos>: {tokenizer.encode('<eos>')}")

print(f"Tamaño vocabulario tokenizer: {len(tokenizer)}")
print(f"Tamaño embeddings modelo: {model.get_input_embeddings().weight.shape[0]}")

Drive ya está montado
Cargando modelo reentrenado desde Google Drive...
Modelo en: cuda:0
Tokens especiales: {'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<|endoftext|>', 'pad_token': '<pad>', 'additional_special_tokens': ['<|talk|>', '<|ax1|>', '<|ax2|>', '<|ax3|>', '<|ax4|>', '<|ax5|>', '<|ax6|>', '<|ax7|>', '<|ax8|>', '<|ax9|>']}
ID de <lyr>: [50261]
ID de <art>: [50260]
ID de <eos>: [50259]
Tamaño vocabulario tokenizer: 50262
Tamaño embeddings modelo: 50262


🔹 Interfaz Gradio para realizar las inferencias

In [ ]:
import gradio as gr
import tempfile

# Función para generar texto con parámetros personalizables
def generate_text(artist, max_new_tokens, temperature, top_k, top_p, repetition_penalty, no_repeat_ngram_size):
    model.eval()
    inputs = tokenizer(f"<bos><art>{artist}<lyr>", return_tensors="pt")
    inputs = {key: value.to(device) for key, value in inputs.items()}
    outputs = model.generate(
        **inputs,
        max_new_tokens=int(max_new_tokens),  # Controla la cantidad de nuevos tokens generados
        temperature=float(temperature),      # Controla la aleatoriedad (solo si do_sample=True)
        top_k=int(top_k),                    # Top-k sampling
        top_p=float(top_p),                  # Top-p (nucleus) sampling
        do_sample=True,                      # Habilita muestreo (permite aleatoriedad - no determinismo)
        pad_token_id=tokenizer.pad_token_id, # Evita advertencias
        eos_token_id=tokenizer.eos_token_id, # Evita advertencias
        repetition_penalty=float(repetition_penalty),
        no_repeat_ngram_size=int(no_repeat_ngram_size),
    )

    text = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Guardar en archivo temporal para descarga
    with tempfile.NamedTemporaryFile(delete=False, mode='w', suffix='.txt') as f:
        f.write(text)
        file_path = f.name

    return text, file_path

# Crear la interfaz de Gradio
interface = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Dropdown(choices=artist_names, label="Artista"),
        gr.Slider(minimum=50, maximum=768, value=512, step=1, label="Máximo de nuevos tokens"),
        gr.Slider(minimum=0.1, maximum=2.0, value=.8, step=0.1, label="Temperature"),
        gr.Slider(minimum=1, maximum=100, value=25, step=1, label="Top K"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.75, step=0.05, label="Top P"),
        gr.Slider(1.0, 2.0, value=1.4, step=0.1, label="Repetition Penalty"),
        gr.Slider(0, 10, value=4, step=1, label="No Repeat N-Gram Size"),
    ],
    theme="default",
    outputs=[
        gr.Textbox(label="Texto generado"),
        gr.File(label="Descargar texto")
    ],
    title=f"Generar una letra de canción (usando el modelo DeepESP/gpt2-spanish)",
    description=f"Elegir un artista y ajusta los parámetros para generar texto con modelo GPT-2 fine-tuneado. Usando {device.upper()}."
)

# Lanzar la interfaz
interface.launch(share=True)